In [1]:
!nvidia-smi

Mon Jun 14 15:32:51 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install wandb 

     |████████████████████████████████| 1.8MB 8.1MB/s 
     |████████████████████████████████| 174kB 48.6MB/s 
     |████████████████████████████████| 102kB 13.4MB/s 
     |████████████████████████████████| 133kB 47.1MB/s 
     |████████████████████████████████| 71kB 11.8MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6502 sha256=0a0fe5e6b9a6de8212cebe0bfc3d1684af3410ab940acb954105de7a582a3eda
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8807 sha256=78d051276890ec4e5bef72389468bdea027fff967c59b603958a76e0b7b5e9f2
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
Successfully built subprocess32 pathtools


In [3]:
import tensorflow as tf 
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
import wandb 
import tensorflow_hub as hub

In [4]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

# Getting the data 

In [ ]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

--2021-06-14 10:08:37--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.20.128, 74.125.197.128, 74.125.195.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.20.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.006s  

2021-06-14 10:08:37 (92.2 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [ ]:
!unzip nlp_getting_started.zip

Archive:  nlp_getting_started.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


# loading the data 


In [ ]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
submission = pd.read_csv('sample_submission.csv')

# Get to know the data 

In [ ]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [ ]:
x_train,x_test,y_train,y_test = train_test_split(train_df.text,train_df.target)

In [ ]:
y_train[:50]

507     0
2174    1
3986    1
6576    0
3419    0
698     0
924     0
16      0
6464    1
862     0
5361    0
6358    1
3440    0
1150    1
6300    0
1380    1
5031    1
344     0
6309    0
6612    0
7102    1
91      1
6946    0
6543    0
5975    0
1605    0
2954    0
1404    0
4640    0
1179    1
262     0
5636    0
2241    0
846     0
5623    0
7457    0
5595    1
4248    1
3079    0
516     1
2171    1
2923    1
29      0
5944    0
3775    0
3685    0
3919    0
1399    1
5026    1
5934    0
Name: target, dtype: int64

# Building the models using tf sequnential api `model_1` , `model_2` , `model_5` 

In [ ]:
wandb.init(project = 'disastrous-nondisastrous',config =  {
    'epochs': 5,
    'name' : "model_1",
    'optimizer' : 'Adam()',
    'learning_rate' : 0.1,
    'loss' : 'binary_crossentropy',
    'training_data' : '100%',
    'metrics': 'accuracy'
})


wandb: Currently logged in as: som (use `wandb login --relogin` to force relogin)


In [ ]:
vectorizer = tf.keras.layers.experimental.preprocessing.TextVectorization(max_tokens=10000,output_mode='int',output_sequence_length=15)

In [ ]:
vectorizer.adapt(x_train.to_list())

In [ ]:
model_1 = tf.keras.Sequential([
                               tf.keras.layers.InputLayer(input_shape= (1,),dtype = tf.string),
                               vectorizer,
                              tf.keras.layers.Embedding(input_dim = 10000, output_dim = 128,input_length=15),
                               tf.keras.layers.GlobalAveragePooling1D(),
                               tf.keras.layers.Dense(1,activation = 'sigmoid')
])

In [ ]:
config = wandb.config 

In [ ]:
# compiling the model 
model_1.compile(loss = 'binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

In [ ]:
model_1_history = model_1.fit(x_train,y_train,
            validation_data = (x_test,y_test),
            epochs = 5,
            steps_per_epoch = len(x_train),
            validation_steps = len(x_test) ,
            callbacks = [wandb.keras.WandbCallback(monitor = 'accuracy',verbose =1 )]
            )

Epoch 1/5
5709/5709 [==============================] - 83s 14ms/step - loss: 0.5082 - accuracy: 0.7592 - val_loss: 0.4491 - val_accuracy: 0.8057


wandb: ERROR Can't save model, h5py returned error: Save or restore weights that is not an instance of `tf.Variable` is not supported in h5, use `save_format='tf'` instead. Got a model or layer TextVectorization with weights [<tensorflow.python.keras.engine.base_layer_utils.TrackableWeightHandler object at 0x7f91210daa90>]


Epoch 00000: accuracy improved from -inf to 0.75915, saving model to /content/wandb/run-20210614_100917-k9fq1upk/files/model-best.h5
Epoch 2/5
5709/5709 [==============================] - 80s 14ms/step - loss: 0.3323 - accuracy: 0.8625 - val_loss: 0.4767 - val_accuracy: 0.7999
Epoch 00001: accuracy improved from 0.75915 to 0.86250
Epoch 3/5
5709/5709 [==============================] - 82s 14ms/step - loss: 0.2562 - accuracy: 0.8996 - val_loss: 0.5245 - val_accuracy: 0.7868
Epoch 00002: accuracy improved from 0.86250 to 0.89963
Epoch 4/5
5709/5709 [==============================] - 82s 14ms/step - loss: 0.2091 - accuracy: 0.9177 - val_loss: 0.5695 - val_accuracy: 0.7810
Epoch 00003: accuracy improved from 0.89963 to 0.91767
Epoch 5/5
5709/5709 [==============================] - 82s 14ms/step - loss: 0.1713 - accuracy: 0.9320 - val_loss: 0.6318 - val_accuracy: 0.7715
Epoch 00004: accuracy improved from 0.91767 to 0.93204


In [ ]:
model_1.evaluate(x_test,y_test)

60/60 [==============================] - 0s 4ms/step - loss: 0.6318 - accuracy: 0.7715


[0.6318238973617554, 0.7715336084365845]

In [ ]:
model_1.predict(x_test)

array([[9.9982208e-01],
       [5.6825977e-02],
       [2.8204718e-01],
       ...,
       [5.7650352e-04],
       [9.9996829e-01],
       [9.9999046e-01]], dtype=float32)

In [ ]:
y_test

1165    1
6172    1
806     0
1300    1
3856    1
       ..
1037    0
4929    0
700     0
6720    1
5296    1
Name: target, Length: 1904, dtype: int64

In [ ]:
wandb.run.finish()

epoch,4
loss,0.17132
accuracy,0.93204
val_loss,0.63182
val_accuracy,0.77153
_runtime,432
_timestamp,1623665789
_step,4
best_accuracy,0.93204
best_epoch,4


epoch,▁▃▅▆█
loss,█▄▃▂▁
accuracy,▁▅▇▇█
val_loss,▁▂▄▆█
val_accuracy,█▇▄▃▁
_runtime,▁▃▄▆█
_timestamp,▁▃▄▆█
_step,▁▃▅▆█


In [ ]:
# next_run = wandb.init(reinit= True)
run = wandb.init(project = 'disastrous-nondisastrous',config ={
    'epochs': 5,
    'name' : "model_2",
    'layers' : 'LSTM',
    'optimizer' : 'Adam()',
    'learning_rate' : 0.1,
    'loss' : 'binary_crossentropy',
    'training_data' : '100%',
    'metrics': 'accuracy'
} )

# model_2 

In [ ]:
model_2 = tf.keras.Sequential([
                               tf.keras.layers.InputLayer(input_shape= (1,),dtype = tf.string),
                               vectorizer,
                               tf.keras.layers.Embedding(input_dim = 10000, output_dim = 128, input_length=15),
                               tf.keras.layers.LSTM(64,return_sequences=True),
                               tf.keras.layers.LSTM(64),  
                               tf.keras.layers.Dense(1,activation = 'sigmoid')
])

In [ ]:
model_2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, 15)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 15, 128)           1280000   
_________________________________________________________________
lstm (LSTM)                  (None, 15, 64)            49408     
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 1,362,497
Trainable params: 1,362,497
Non-trainable params: 0
_________________________________________________________________


In [ ]:
config = wandb.config

In [ ]:
model_2.compile(
                 loss = tf.keras.losses.binary_crossentropy,
                 optimizer = tf.keras.optimizers.Adam(),
                 metrics = ['accuracy'])

In [ ]:
model_2_history = model_2.fit(
    x_train,y_train,
    validation_data = (x_train,y_train),
    epochs = 5 , 
    callbacks = [wandb.keras.WandbCallback(monitor = 'accuracy',verbose = 1)]
)

Epoch 1/5
179/179 [==============================] - 12s 37ms/step - loss: 0.5242 - accuracy: 0.7446 - val_loss: 0.3550 - val_accuracy: 0.8642


wandb: ERROR Can't save model, h5py returned error: Save or restore weights that is not an instance of `tf.Variable` is not supported in h5, use `save_format='tf'` instead. Got a model or layer TextVectorization with weights [<tensorflow.python.keras.engine.base_layer_utils.TrackableWeightHandler object at 0x7f91210daa90>]


Epoch 00000: accuracy improved from -inf to 0.74461, saving model to /content/wandb/run-20210614_101803-309e5732/files/model-best.h5
Epoch 2/5
179/179 [==============================] - 6s 31ms/step - loss: 0.3139 - accuracy: 0.8728 - val_loss: 0.1922 - val_accuracy: 0.9443
Epoch 00001: accuracy improved from 0.74461 to 0.87283
Epoch 3/5
179/179 [==============================] - 6s 31ms/step - loss: 0.2043 - accuracy: 0.9298 - val_loss: 0.1352 - val_accuracy: 0.9588
Epoch 00002: accuracy improved from 0.87283 to 0.92976
Epoch 4/5
179/179 [==============================] - 6s 32ms/step - loss: 0.1515 - accuracy: 0.9494 - val_loss: 0.0899 - val_accuracy: 0.9711
Epoch 00003: accuracy improved from 0.92976 to 0.94938
Epoch 5/5
179/179 [==============================] - 6s 31ms/step - loss: 0.1048 - accuracy: 0.9627 - val_loss: 0.0610 - val_accuracy: 0.9781
Epoch 00004: accuracy improved from 0.94938 to 0.96269


In [ ]:
wandb.__version__

'0.10.32'

In [ ]:
print('hello')

hello


In [ ]:
run.finish()

epoch,4
loss,0.10481
accuracy,0.96269
val_loss,0.06098
val_accuracy,0.9781
_runtime,46
_timestamp,1623665929
_step,4
best_accuracy,0.96269
best_epoch,4


epoch,▁▃▅▆█
loss,█▄▃▂▁
accuracy,▁▅▇██
val_loss,█▄▃▂▁
val_accuracy,▁▆▇██
_runtime,▁▃▅▆█
_timestamp,▁▃▅▆█
_step,▁▃▅▆█


# model 5 

In [ ]:
wandb.init(project= 'disastrous-nondisastrous',
           config = {
               'epochs': 5,
    'name' : "model_5",
    'layers' : 'Conv1D',
    'optimizer' : 'tf.keras.optimizers.Adam()',
    'learning_rate' : 0.1,
    'loss' : 'binary_crossentropy',
    'training_data' : '100%',
    'metrics': ['accuracy']
           })

In [ ]:
config= wandb.config

In [ ]:
model_5 = tf.keras.Sequential([
                               tf.keras.layers.InputLayer(input_shape = (1,),dtype = tf.string),
                               vectorizer,
                               tf.keras.layers.Embedding(input_dim = 10000,output_dim = 128,input_length= 15 ),
                               tf.keras.layers.Conv1D(32,4,1,padding = 'same',activation = 'relu'),
                               tf.keras.layers.GlobalMaxPool1D(),
                               tf.keras.layers.Dense(1,activation = 'sigmoid')

])

In [ ]:
config

{'epochs': 5, 'name': 'model_5', 'layers': 'Conv1D', 'optimizer': '<tensorflow.python.keras.optimizer_v2.adam.Adam object at 0x7f90b2999c50>', 'learning_rate': 0.1, 'loss': 'binary_crossentropy', 'training_data': '100%', 'metrics': ['accuracy']}

In [ ]:
model_5.compile(loss = config.loss,
                optimizer = tf.keras.optimizers.Adam() , 
                metrics = config.metrics)

In [ ]:
model_5_history = model_5.fit(
    x_train,y_train,
    validation_data = (x_test,y_test),
    epochs = 5 , 
    callbacks = [wandb.keras.WandbCallback(verbose = 1)]
)

Epoch 1/5
179/179 [==============================] - 17s 19ms/step - loss: 0.5839 - accuracy: 0.6970 - val_loss: 0.4700 - val_accuracy: 0.7978


wandb: ERROR Can't save model, h5py returned error: Save or restore weights that is not an instance of `tf.Variable` is not supported in h5, use `save_format='tf'` instead. Got a model or layer TextVectorization with weights [<tensorflow.python.keras.engine.base_layer_utils.TrackableWeightHandler object at 0x7f91210daa90>]


Epoch 00000: val_loss improved from inf to 0.46997, saving model to /content/wandb/run-20210614_102709-3lzsijfk/files/model-best.h5
Epoch 2/5
179/179 [==============================] - 3s 18ms/step - loss: 0.3600 - accuracy: 0.8494 - val_loss: 0.4504 - val_accuracy: 0.7967
Epoch 00001: val_loss improved from 0.46997 to 0.45035
Epoch 3/5
179/179 [==============================] - 3s 17ms/step - loss: 0.2217 - accuracy: 0.9207 - val_loss: 0.5083 - val_accuracy: 0.7831
Epoch 4/5
179/179 [==============================] - 3s 17ms/step - loss: 0.1373 - accuracy: 0.9538 - val_loss: 0.5650 - val_accuracy: 0.7794
Epoch 5/5
179/179 [==============================] - 3s 17ms/step - loss: 0.0976 - accuracy: 0.9665 - val_loss: 0.6288 - val_accuracy: 0.7752


In [ ]:
model_5.evaluate(x_test,y_test)

60/60 [==============================] - 0s 4ms/step - loss: 0.6288 - accuracy: 0.7752


[0.6287891864776611, 0.7752100825309753]

In [ ]:
run.finish()

# fine tuning TF Hub Universal Sentence Encoder model 

In [ ]:
wandb.init(project= 'disastrous-nondisastrous',
           config = {
               'epochs': 5,
                'name' : "tf_hub_model",
                'layers' : 'USE,Conv1D',
    'optimizer' : 'tf.keras.optimizers.Adam()',
    'learning_rate' : 0.1,
    'loss' : 'binary_crossentropy',
    'training_data' : '100%',
    'metrics': ['accuracy']
           })

In [ ]:
sentence_encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                        input_shape =[],
                                         dtype=tf.string,
                                        trainable=False)

In [ ]:
model_transfer_learning_not_fine_tuned = tf.keras.Sequential([
                                                              sentence_encoder_layer,
                                                              # tf.keras.layers.Conv1D(64,4,1,'same',activation ='relu'),
                                                              tf.keras.layers.Dense(64,activation = 'relu'),
                                                              # tf.keras.layers.GlobalAveragePooling1D(),
                                                              tf.keras.layers.Dense(1,activation = 'sigmoid')
])

In [ ]:
wandb.init(project= 'disastrous-nondisastrous',
           config = {
               'epochs': 5,
                'name' : "tf_hub_model_fine_tuned",
                'layers' : 'USE,dense_2 layers ',
    'optimizer' : 'Adam',
    'learning_rate' : 0.1,
    'loss' : 'binary_crossentropy',
    'training_data' : '100%',
    'metrics': ['accuracy']
           })

In [ ]:
config = wandb.config

In [ ]:
model_transfer_learning_not_fine_tuned.compile(optimizer = tf.keras.optimizers.Adam(),
                                               loss = config.loss,
                                               metrics = config.metrics)

In [ ]:
without_fine_tune_history = model_transfer_learning_not_fine_tuned.fit(x_train,y_train,
                                                                       validation_data = (x_test,y_test),
                                                                       epochs = 5 , 
                                                                       callbacks = [wandb.keras.WandbCallback(verbose =1 , monitor = 'accuracy')])

Epoch 1/5
179/179 [==============================] - 17s 56ms/step - loss: 0.5247 - accuracy: 0.7676 - val_loss: 0.4338 - val_accuracy: 0.8114
Epoch 00000: accuracy improved from -inf to 0.76756, saving model to /content/wandb/run-20210614_114059-2gsp2te5/files/model-best.h5
Epoch 2/5
179/179 [==============================] - 5s 29ms/step - loss: 0.4215 - accuracy: 0.8126 - val_loss: 0.4179 - val_accuracy: 0.8146
Epoch 00001: accuracy improved from 0.76756 to 0.81258, saving model to /content/wandb/run-20210614_114059-2gsp2te5/files/model-best.h5
Epoch 3/5
179/179 [==============================] - 5s 29ms/step - loss: 0.4051 - accuracy: 0.8210 - val_loss: 0.4152 - val_accuracy: 0.8151
Epoch 00002: accuracy improved from 0.81258 to 0.82098, saving model to /content/wandb/run-20210614_114059-2gsp2te5/files/model-best.h5
Epoch 4/5
179/179 [==============================] - 5s 31ms/step - loss: 0.3953 - accuracy: 0.8271 - val_loss: 0.4137 - val_accuracy: 0.8225
Epoch 00003: accuracy impr

In [ ]:
model_transfer_learning_not_fine_tuned.evaluate(x_test,y_test)

60/60 [==============================] - 1s 23ms/step - loss: 0.4127 - accuracy: 0.8220


[0.41266337037086487, 0.8219537734985352]

In [ ]:
model_transfer_learning_not_fine_tuned.summary()

Model: "sequential_33"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_10 (KerasLayer)  (None, 512)               256797824 
_________________________________________________________________
dense_41 (Dense)             (None, 64)                32832     
_________________________________________________________________
dense_42 (Dense)             (None, 1)                 65        
Total params: 256,830,721
Trainable params: 32,897
Non-trainable params: 256,797,824
_________________________________________________________________


In [ ]:
sentence_encoder_layer.trainable = True

In [ ]:
model_transfer_learning_not_fine_tuned.compile(loss = config.loss,
                                               optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0001),
                                               metrics = config.metrics)

In [ ]:
without_fine_tune_history.epoch[-1]

4

In [ ]:
fine_tuned_model = model_transfer_learning_not_fine_tuned.fit(x_train,y_train,
                                                              validation_data = (x_test,y_test),
                                                              epochs = 10 , 
                                                              initial_epoch = without_fine_tune_history.epoch[-1],
                                                              callbacks = [wandb.keras.WandbCallback()])

Epoch 5/10
179/179 [==============================] - 32s 129ms/step - loss: 0.3765 - accuracy: 0.8366 - val_loss: 0.4085 - val_accuracy: 0.8172
Epoch 6/10
179/179 [==============================] - 21s 115ms/step - loss: 0.3317 - accuracy: 0.8593 - val_loss: 0.4074 - val_accuracy: 0.8230
Epoch 7/10
179/179 [==============================] - 21s 116ms/step - loss: 0.2886 - accuracy: 0.8825 - val_loss: 0.4198 - val_accuracy: 0.8146
Epoch 8/10
179/179 [==============================] - 21s 116ms/step - loss: 0.2421 - accuracy: 0.9063 - val_loss: 0.4289 - val_accuracy: 0.8114
Epoch 9/10
179/179 [==============================] - 20s 114ms/step - loss: 0.1949 - accuracy: 0.9278 - val_loss: 0.4435 - val_accuracy: 0.8109
Epoch 10/10
179/179 [==============================] - 21s 115ms/step - loss: 0.1461 - accuracy: 0.9518 - val_loss: 0.4836 - val_accuracy: 0.7957


In [ ]:
predictions = model_transfer_learning_not_fine_tuned.predict(x_test)

In [ ]:
len(y_test.to_numpy())

1904

In [ ]:
tf.cast(tf.squeeze(tf.round(predictions)),dtype = tf.int32)

<tf.Tensor: shape=(1904,), dtype=int32, numpy=array([1, 0, 0, ..., 0, 1, 1], dtype=int32)>

In [ ]:
wandb.log({"conf_mat" : wandb.plot.confusion_matrix(probs=None,
                        y_true=y_test.to_numpy(), preds=np.array(tf.cast(tf.squeeze(tf.round(predictions)),dtype = tf.int32)),
                        class_names=['non_disaster','disaster'])})

In [ ]:
run.finish()